In [ ]:
!pip install rouge datasets==1.4.1 transformers==4.4.2

## preprocess data

In [4]:
## data process 
import pandas as pd
from datasets import load_dataset

df_article_summary_full = pd.read_parquet('../../../data-v1/article_summary.parquet', engine='pyarrow')
df_article_summary_full[['original_text','summary']].to_csv('./total.csv',index=False)



dataset = load_dataset('csv', data_files='./total.csv',split='train')
dataset =  dataset.rename_column("original_text", "article")
dataset =  dataset.rename_column("summary", "summarization")
#train test split
dataset = dataset.train_test_split(test_size=0.1)

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
Using custom data configuration default-515c8d33f7b5171f


Dataset csv downloaded and prepared to /home/ec2-user/.cache/huggingface/datasets/csv/default-515c8d33f7b5171f/0.0.0/2a88c45fed596f9421a2e7f74ab1a3cd012ef75210a5dc1950e8d60ca8d9c66c. Subsequent calls will reuse this data.


In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'summarization'],
        num_rows: 13500
    })
    test: Dataset({
        features: ['article', 'summarization'],
        num_rows: 1500
    })
})

In [6]:
import os

path = './demo_data/SUMMARY.hk01'
os.makedirs(path, exist_ok=True)
print("The new directory is created!")

#dataset['train'].to_json(os.path.join(path,'train.json'),orient = 'records')
#dataset['test'].to_json(os.path.join(path,'test.json'), orient = 'records')

# use csv file to test 
dataset['train'].to_csv(os.path.join(path,'train.csv'))
dataset['test'].to_csv(os.path.join(path,'test.csv'))
dataset['test'].to_csv(os.path.join(path,'dev.csv'))

The new directory is created!


4327736

## run train

In [7]:
#!python run_gen.py --model_path 'fnlp/cpt-base' --dataset adgen --data_dir demo_data
!python run_gen_v2.py --model_path 'fnlp/cpt-base' --dataset hk01 --data_dir demo_data --epoch '1'

Traceback (most recent call last):
  File "run_gen_v2.py", line 11, in <module>
    import torch
ModuleNotFoundError: No module named 'torch'


In [14]:
#finetune/modeling_cpt.py
from transformers import BertTokenizer

tokenizer=BertTokenizer.from_pretrained('output/adgen/2')
ids=tokenizer.encode("[[类型, 上衣], [版型, 宽松], [风格, 复古], [图案, 复古], [衣样式, 针织衫]]")
tokenizer.decode(ids,skip_special_tokens=True)

'[ [ 类 型, 上 衣 ], [ 版 型, 宽 松 ], [ 风 格, 复 古 ], [ 图 案, 复 古 ], [ 衣 样 式, 针 织 衫 ] ]'

In [6]:
!ls

bleu_metric.py	output	     README.md	  run_gen.py  Untitled.ipynb
demo_data	__pycache__  run_bleu.py  runs	      utils.py


In [10]:
!ls output/adgen/2

all_results.json   special_tokens_map.json  trainer_state.json	vocab.txt
config.json	   test_generations.txt     training_args.bin
pytorch_model.bin  tokenizer_config.json    train_results.json


In [21]:
!python run_bleu.py

0.03574327267496428


In [22]:
import os
from utils import load_json
from transformers import BertTokenizer
from bleu_metric import Metric

#需要计算bleu的文本文件所在的目录
arch='output'
tokenizer=BertTokenizer.from_pretrained('output/adgen/2')
dataset='adgen'
test_set=load_json('demo_data/SUMMARY.adgen/test.json')

labels=[]
for data in test_set:
    ids=tokenizer.encode(data['summarization'])
    labels.append(tokenizer.decode(ids,skip_special_tokens=True))
    
labels=[[label.strip().split(' ')] for label in labels]


In [25]:
test_set[0]

{'summarization': '这款阔腿裤，整体设计简约利落，时尚的阔腿款式带来鲜明的几何设计美感，褪去传统装束的厚重与臃肿，更具轻盈美感。 搭配七分裤长修饰出挺拔的腿部线条，气质的格纹图案不显单调，尽显女性优雅气质。 斜门襟设计潮流出众，让你时刻保持动人的女性风采。',
 'article': '[[类型, 裤], [风格, 简约], [风格, 潮], [图案, 格子], [图案, 几何], [图案, 线条], [裤长, 七分裤], [裤型, 阔腿裤]]'}

In [40]:
model_inputs = tokenizer(test_set[0]['article'], max_length=128, padding="max_length", truncation=True)

In [41]:
model_inputs

{'input_ids': [101, 138, 138, 5102, 1798, 117, 6175, 140, 117, 138, 7599, 3419, 117, 5042, 5276, 140, 117, 138, 7599, 3419, 117, 4060, 140, 117, 138, 1745, 3428, 117, 3419, 2094, 140, 117, 138, 1745, 3428, 117, 1126, 862, 140, 117, 138, 1745, 3428, 117, 5296, 3340, 140, 117, 138, 6175, 7270, 117, 673, 1146, 6175, 140, 117, 138, 6175, 1798, 117, 7333, 5597, 6175, 140, 140, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1

## 推理

In [108]:
import sys
sys.path.append('../')

from modeling_cpt import CPTModel, CPTForConditionalGeneration

model = CPTForConditionalGeneration.from_pretrained("output/adgen/2")
tokenizer = BertTokenizer.from_pretrained('output/adgen/2')

#model_inputs = tokenizer(test_set[0]['article'], max_length=128, padding="max_length", truncation=True,return_tensors="pt")

input:  [[类型, 裤], [风格, 简约], [风格, 潮], [图案, 格子], [图案, 几何], [图案, 线条], [裤长, 七分裤], [裤型, 阔腿裤]]
约 ， 简 约 自 然 简 约 的 设 计 感 强 烈 的 感 觉, 简 单 的 复 古 典 主 义 感 觉 到 极 致 感 觉 的 美 感,, 图 案, 线 条 。 。 。 图 案 的 几 何 图 案 ， 格 子,, [ 风 格, 格 子 线 条, 几 何 线 条 ， 图 案 、 线 条 图 案 。 。 [SEP]


In [113]:
%%time
#input_text = "[[类型, 裤], [风格, 简约], [风格, 潮], [图案, 格子], [图案, 几何], [图案, 线条], [裤长, 七分裤], [裤型, 阔腿裤]]"
input_text = "自有紀錄以來，巴約掛毯已在法國「生活」了950年，從未離開過「家門」。有報道指，法國總統馬克龍答應首次將巴約掛氈借出海外，而獲得這份「借來的厚禮」，就是正忙於脫歐的英國。按官方說法，這次「掛毯外交」旨在展示英法友好關係。但在友誼背後，卻揭示馬克龍「一啖砂糖一啖屎」、軟硬兼施的外交哲學SPLIT中國有一幅傳世名畫《清明上河圖》，法國也有，而它叫作「巴約掛氈」（Bayeux Tapestry）。 這幅近千年歷史的刺鏽畫上11世紀諾曼征服英格蘭的黑斯廷斯戰役（Battle of Hastings），極具歷史價值。 "
print("input: ",input_text)

inputs = tokenizer(input_text, return_tensors="pt")

prompt_length = len(tokenizer.decode(inputs['input_ids'][0]))
outputs = model.generate(inputs['input_ids'], max_length=250, do_sample=True, top_p=0.95, top_k=60)
generated = tokenizer.decode(outputs[0])[prompt_length+1:]

print("result: ",generated)

input:  自有紀錄以來，巴約掛毯已在法國「生活」了950年，從未離開過「家門」。有報道指，法國總統馬克龍答應首次將巴約掛氈借出海外，而獲得這份「借來的厚禮」，就是正忙於脫歐的英國。按官方說法，這次「掛毯外交」旨在展示英法友好關係。但在友誼背後，卻揭示馬克龍「一啖砂糖一啖屎」、軟硬兼施的外交哲學SPLIT中國有一幅傳世名畫《清明上河圖》，法國也有，而它叫作「巴約掛氈」（Bayeux Tapestry）。 這幅近千年歷史的刺鏽畫上11世紀諾曼征服英格蘭的黑斯廷斯戰役（Battle of Hastings），極具歷史價值。 
result:   ， 法 国 国 王 的 马 克 龍 是 英 国 王 朝 的 王 朝 军 队 的 一 次 巴 约 [SEP]
CPU times: user 18.4 s, sys: 0 ns, total: 18.4 s
Wall time: 5.31 s
